# 10. 프로젝트 : 단어 Level로 번역기 업그레이드하기


### 라이브러리를 준비하자

In [1]:
import tensorflow

print(tensorflow.__version__)

2.6.0


## Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)

### 1. 구두점(Punctuation)을 단어와 분리해주세요.

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 194513


,eng,fra,cc
185554,Tom drank three bottles of wine by himself las...,Tom a bu trois bouteilles de vin à lui tout se...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
179362,I can't figure out why nobody comes here anymore.,Je ne comprends pas pourquoi personne ne vient...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
145536,That university was my first choice.,Cette université était mon premier choix.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
138699,I don't feel like going to bed yet.,Je n'ai pas encore envie d'aller me coucher.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
91618,A child is crying somewhere.,Il y a quelque part un enfant qui pleure.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [4]:
lines = lines[['eng', 'fra']][:33000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
25077,Let's go that way.,Allons de ce côté.
4867,I feel faint.,Je me sens pris de malaise.
31713,The secret got out.,Le secret fut révélé.
21973,You look perfect.,Vous avez l'air parfait.
32641,Was it all a dream?,Tout était-il un rêve ?


In [5]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
20565,They're all dead.,\t Ils sont tous morts. \n
28402,Does your dog bite?,\t Votre chien mord-il ? \n
27886,You're very brave.,\t Vous êtes très braves. \n
5094,I was warned.,\t J'ai été prévenu. \n
15119,Leave our house.,\t Quitte notre maison ! \n


In [50]:
import os, re 
import numpy as np
import tensorflow as tf

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence



In [51]:
import random


eng_list=lines['eng']
eng_list=eng_list.values.tolist()
type(eng_list)
eng_str=eng_list[3000:3100]
preprocess_sentence(eng_str)

AttributeError: 'list' object has no attribute 'lower'

In [62]:
for i in range(100):
    eng_str=preprocess_sentence(eng_list[i])
eng_str

'<start> hop in . <end>'

In [59]:
preprocess_sentence(a.eng)

AttributeError: 'list' object has no attribute 'eng'

In [ ]:
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

In [ ]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]